In [ ]:
import google.generativeai as genai
import base64
from PIL import Image
from io import BytesIO
import requests
import os
import json
import re
import pandas as pd
from vertexai.preview.generative_models import Part
                

In [ ]:

genai.configure(api_key="")


In [ ]:
model = genai.GenerativeModel("gemini-2.0-flash-lite")


In [ ]:
def encode_image(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def escape_inches(st):
    # Add a single backslash before a " that follows a digit
    return re.sub(r'(?<=\d)"', '\\"', st)


In [ ]:
img_path = '../Tell2Design Data/General Data/floorplan_image/'
json_path = '../Tell2Design Data/t2d.json'

In [ ]:
with open(json_path) as f:
    data = json.load(f)

In [ ]:
data['77414.png']

In [ ]:
'''
img_path = path + 'floorplan_image/' + '25.png'
text_path = path + 'human_annotated_tags/' + '25.txt'
'''

In [ ]:
'''
example_images_floor = [
    
    {"path": img_path, 
    'living': [[137, 66], [137, 95], [136, 96], [95, 96], [95, 120], [94, 121], [74, 121], [74, 138], [141, 138], [142, 137], [144, 139], [144, 185], [153, 185], [154, 184], [156, 186], [156, 190], [192, 190], [192, 125], [161, 125], [160, 124], [160, 66]],
    'bath': [[164, 96], [164, 121], [192, 121], [192, 96]],
    'common': [[164, 66], [164, 92], [206, 92], [206, 66]],
    'master': [[95, 142], [95, 190], [128, 190], [128, 186], [130, 184], [131, 185], [140, 185], [140, 142]],
    'kitchen': [[196, 96], [196, 107], [206, 107], [206, 96]]}
    'description':'The bathroom is located on the east side, It is next to living and kitchen. The common room is located on the south west corner, It is next to living and master. The kitchen is located on the north east corner, It is next to living and bath. The living room is located on the north side, It is in front of all other rooms. The master room is located on the south side, It is next to living and near to common. The storage room is located on the east side,it is inside southeast corner of kitchen.'}
    
    
    living_0 polygon: [[61, 122], [61, 217], [125, 217], [125, 175], [126, 174], [195, 174], [195, 156], [179, 156], [178, 155], [178, 122]]
    bath_0 polygon: [[168, 178], [168, 217], [195, 217], [195, 178]]
    common_0 polygon: [[129, 178], [129, 217], [164, 217], [164, 178]]
    master_0 polygon: [[154, 58], [154, 118], [195, 118], [195, 58]]
    kitchen_0 polygon: [[182, 122], [182, 152], [195, 152], [195, 122]]
    balcony_0 polygon: [[154, 39], [154, 54], [195, 54], [195, 39]]
        ]
encoded_examples = [{"base64": encode_image(e["path"]), "living":e["living"], "bath":e["bath"], "common":e["common"], "master":e["master"], "kitchen":e["kitchen"], "description":['description']} for e in example_images_floor]
'''

In [ ]:
system_prompt = {
    "role": "user",
    "parts": [
        {
            "text": """You are a top-level architect with years of experience reading detailed architectural site plans. You analyze floor plans with precision, explain their structure, and extract room locations and polygon coordinates accurately. You can perform multiple tasks in one response. The final description should be detailed. 

            <color-scheme>
            Use this exact color-to-room mapping:
            - balcony:      (107, 142, 35)
            - common room:  (255, 215, 0)
            - bathroom:     (173, 216, 230)
            - kitchen:      (240, 128, 128)
            - master room:  (255, 165, 0)
            - living room:  (238, 232, 170)
            - entrance:     (255, 0, 0)
            - dining room:  (218, 112, 214)
            - storage:      (221, 160, 221)
            </color-scheme>

            <assumptions>
            - The top of the image is North.
            - Image size is 256×256 pixels.
            - If multiple rooms share a type, label them type_1, type_2, … (e.g., common_1, common_2).
            </assumptions>

            <task>
            Return ONLY a JSON object with THREE sections: "reasoning", "facts", and "description". 
            Everything in "facts" and "description" MUST cite provenance via a "support" array of integers that reference indices in "reasoning.steps".

            1) reasoning
               - steps: An ordered list of short, atomic observations from the image (e.g., color segmentation result, boundaries traced, centroids computed, adjacency detected).
               - For every task how you reached to the answer.
               - claims: Normalize key statements supported by the image. Each claim must include:
                 { "type": "orientation"|"adjacency"|"count"|"geometry",
                   "subject": "<room_id or 'rooms'>",
                   "relation": "north_of"|"touches"|"equals"|"aspect_ratio_approx"|...,
                   "object": "<room_id>" (if applicable),
                   "value": "<number or ratio string>" (if applicable),
                   "tolerance": <number> (if applicable),
                   "support": [<indices into reasoning.steps>] }

            2) facts  (image-grounded measurements you extracted)
               - rooms: [{ "id":"<room_id>", "color_rgb":[r,g,b], "support":[...] }, ...]
               - geometry:
                   {
                     "image_size_px": { "w":256, "h":256, "support":[...] },
                     "polygons_px": [ { "id":"<room_id>", "points":[[x,y],...], "source":"color_segmentation", "support":[...] }, ... ],
                     "bboxes_px":   [ { "id":"<room_id>", "x":x, "y":y, "w":w, "h":h, "support":[...] }, ... ],
                     "centroids_px":[ { "id":"<room_id>", "cx":cx, "cy":cy, "support":[...] }, ... ],
                     "adjacency":   [ { "a":"<room_id>","b":"<room_id>","touches":true, "method":"edge_or_bbox_touch", "support":[...] }, ... ]
                   }

            3) description
               - A **single continuous text block** (string) giving a clear, human-readable description of:
                 - Room arrangement and positions (north, south, center, etc.)
                 - Connectivity (which rooms are adjacent or connected)
                 - Geometry details (orientation, relative size/shape)
                 - Room count
               - Must be written in complete sentences, not bullet points.
               - Must be based entirely on supported reasoning steps and facts; include `"support":[...]` listing the indices of reasoning steps used.

            </task>

            <constraints>
            - NO SHORTCUTS: "description" must not introduce any information not present in "reasoning" and "facts".
            - PROVENANCE REQUIRED: Every item outside "reasoning" must include a non-empty "support" array of integers indexing "reasoning.steps".
            - If something cannot be supported, leave it out rather than guessing.
            - Output valid JSON only. No markdown fences, no extra text outside the JSON.
            </constraints>

            """
        }
    ]
}


In [ ]:
system_prompt = {
    "role": "user",
    "parts": [
        {
            "text": """You are a top-level architect with years of experience reading detailed architectural site plans. You analyze floor plans with precision, explain their structure, and extract room locations and polygon coordinates accurately. You can perform multiple tasks in one response. The final description should be detailed. 

            <color-scheme>
            Use this exact color-to-room mapping:
            - balcony:      (107, 142, 35)
            - common room:  (255, 215, 0)
            - bathroom:     (173, 216, 230)
            - kitchen:      (240, 128, 128)
            - master room:  (255, 165, 0)
            - living room:  (238, 232, 170)
            - entrance:     (255, 0, 0)
            - dining room:  (218, 112, 214)
            - storage:      (221, 160, 221)
            </color-scheme>

            <assumptions>
            - The top of the image is North.
            - Image size is 256×256 pixels.
            - If multiple rooms share a type, label them type_1, type_2, … (e.g., common_1, common_2).
            </assumptions>

            <task>
            Return ONLY a JSON object with THREE sections: "reasoning", "coords", and "description". 
            Everything in "facts" and "description" MUST cite provenance via a "support" array of integers that reference indices in "reasoning.steps".

            1) reasoning
                - steps: An ordered list of steps that help you arrive at the coordinates and description.
                - For each task, explain how you arrived at the answer.
               

            2) Coordinates or measurements you extracted
               - geometry:
                   {
                     "coords:" [living: [[x1, y1], [x2, y2], ..., [xn, yn]],
                      "bathroom:" [[x1, y1], ..., [xn, yn]],
                      kitchen: [[x1, y1], ..., [xn, yn]],
                      ...
                      ...
                      ...]
                    }

            3) description
               - A **single continuous text block** (string) giving a clear, human-readable description of:
                 - Room arrangement and positions (north, south, center, etc.)
                 - Connectivity (which rooms are adjacent or connected)
                 - Geometry details (orientation, relative size/shape)
                 - Room count
               - Must be written in complete sentences, not bullet points.
               - Add other details which you feel appropriate for the floor plan.

            </task>

            <constraints>
            - Do not repeat boilerplate steps. Your reasoning should be based on the specific layout and room positions in this image.
            - Output valid JSON only. No markdown fences, no extra text outside the JSON.
            </constraints>

            """
        }
    ]
}
        {"text": "Now describe the map and extract coordinates of rooms from map. Also give step-by-step reasoning for answer. Explain step-by-step how you reach to the description and coordinates of map elements. "}


In [ ]:
system_prompt = {
    "role": "user",
    "parts": [
        {
            "text": """You are a top-level architect with years of experience reading detailed architectural site plans. Give chain of thoughts for the task and answer also. 

            <color-scheme>
            Use this exact color-to-room mapping:
            - balcony:      (107, 142, 35)
            - common room:  (255, 215, 0)
            - bathroom:     (173, 216, 230)
            - kitchen:      (240, 128, 128)
            - master room:  (255, 165, 0)
            - living room:  (238, 232, 170)
            - entrance:     (255, 0, 0)
            - dining room:  (218, 112, 214)
            - storage:      (221, 160, 221)
            </color-scheme>

            <assumptions>
            - The top of the image is North.
            - Image size is 256×256 pixels.
            - If multiple rooms share a type, label them type_1, type_2, … (e.g., common_1, common_2).
            </assumptions>
            
            <task>
            1. What rooms you crossed from entrace to kitchen?
            2. Do kitchen and living room share boundary?
            
            <output format>
            "CoT:" ...
            "Answer:" ...
            
            </output format>
            """
        }
    ]
}


In [ ]:
system_prompt = {
    "role": "user",
    "parts": [
        {
            "text": """You are a top-level architect with years of experience reading detailed architectural site plans.

            <color-scheme>
            Use this exact color-to-room mapping:
            - balcony:      (107, 142, 35)
            - common room:  (255, 215, 0)
            - bathroom:     (173, 216, 230)
            - kitchen:      (240, 128, 128)
            - master room:  (255, 165, 0)
            - living room:  (238, 232, 170)
            - entrance:     (255, 0, 0)
            - dining room:  (218, 112, 214)
            - storage:      (221, 160, 221)
            </color-scheme>

            <assumptions>
            - The top of the image is North.
            - Image size is 256×256 pixels.
            - If multiple rooms share a type, label them type_1, type_2, … (e.g., common_1, common_2).
            </assumptions>
            
            <task>
            1. Provide a step-by-step reasoning process for floor plan description.
            </task>
            
           
            
            <output format>
            "CoT:" ...
            "description:" ...
            </output format>
            
            
            """
            
        }
    ]
}


In [ ]:
system_prompt = {
    "role": "user",
    "parts": [
        {
            "text": """You are a top-level architect with years of experience reading detailed architectural site plans.

            <color-scheme>
            Use this exact color-to-room mapping:
            - balcony:      (107, 142, 35)
            - common room:  (255, 215, 0)
            - bathroom:     (173, 216, 230)
            - kitchen:      (240, 128, 128)
            - master room:  (255, 165, 0)
            - living room:  (238, 232, 170)
            - entrance:     (255, 0, 0)
            - dining room:  (218, 112, 214)
            - storage:      (221, 160, 221)
            </color-scheme>

            <assumptions>
            - The top of the image is North.
            - Image size is 256×256 pixels.
            - If multiple rooms share a type, label them type_1, type_2, … (e.g., common_1, common_2).
            </assumptions>
            
            <task>
            1. Provide a step-by-step reasoning process for your final analysis of the floor plan.
            2. Provide a separate step-by-step reasoning process for determining each room’s polygon coordinates.
            </task>
            
            <example>
            The description or analysis can be as: The bathroom at the northwest corner of the apartment.  The bathroom is adjacent to the kitchen and living room. 
            A kitchen is located on the north side of the apartment.  The kitchen is  positioned next to the bathroom, common room, and balcony.
            A common room on the north side of the apartment. The common room has an en-suite bathroom and be adjacent to the kitchen and master room.
            A master room on the south side of the apartment. The master room is an en-suite bathroom and be positioned next to the common room and balcony.
            Balcony is on the south side of the apartment, The balcony is  adjacent to the kitchen, master room, and living room.'
            You can talk about :
            Room arrangement and positions (north, south, center, etc.)
                 - Connectivity (which rooms are adjacent or connected)
                 - Geometry details (orientation, relative size/shape)
                 - Room count
            ...
            ...
            </example>
            
            <output format>
            "CoT:" ...
            "Analysis:" ...
            "coords:" ...
            </output format>
            
            <important>
            Do not copy paste the boilerplate example in the prediction. 
            </important>
            """
            
        }
    ]
}


In [ ]:
def save_pred(output, gt, path):
    output = output.replace('"', "")
    output = output.replace('*', "")
    output = output.replace('```json', "")
    output = output.replace('```', "")

    reason_start = re.search(r'Reason', output, re.DOTALL).span()[0]
    reason_end = re.search(r'Reason', output, re.DOTALL).span()[1]
    answer_start = re.search(r'Answer', output, re.DOTALL).span()[0]
    answer_end = re.search(r'Answer', output, re.DOTALL).span()[1]
    coords_start = re.search(r'coords', output, re.DOTALL).span()[0]
    coords_end = re.search(r'coords', output, re.DOTALL).span()[1]

    reason_text = output[reason_end:answer_start]
    reason_text = reason_text.replace("[", "")
    reason_text = reason_text.replace("]", "")
    reason_text = reason_text.replace("'", "")
    reason_text = reason_text.replace(":", "")

    reason_lines = [line.strip() for line in reason_text.strip().split("\n") if line.strip()]

    answer_text = output[answer_end:coords_start]
    answer_str = answer_text.replace("Answer:", "").strip()
    answer_text = answer_text.replace("[", "")
    answer_text = answer_text.replace("]", "")
    answer_text = answer_text.replace("'", "")
    answer_text = answer_text.replace(":", "")

    coords_text = output[coords_end:]
    coords_str = coords_text.replace("Answer:", "").strip()

    result = {
            "Reason": reason_lines,
            "Answer": answer_str,
            "coords": coords_str,
            "gt":gt
        }
    with open(path, "w") as f:
        json.dump(result, f, indent=2)
    
    

In [ ]:
for i, each in enumerate(data.keys()):
    print(i)
    #if i < 370:
    #    continue
    
    img = img_path + each
    messages = [system_prompt]
    test_img = encode_image(img)
    messages.append({
    "role": "user",
    "parts": [
        {"mime_type": "image/jpeg", "data": test_img},
        {"text": "Give chain of thoughts and answer the questions for this floor plan."}
        ]
    })
    
    response = model.generate_content(messages)
    output = response.text
    path = save_path + each.split('.txt')[0] + '.json'
    save_pred(output, content, path)
    
    

In [ ]:
output

In [ ]:
split=output.split('\n')

In [ ]:
split[2].strip()

In [ ]:
reason_start = re.search(r"Reason:", output, re.DOTALL).span()[0]
reason_end = re.search(r"Reason:", output, re.DOTALL).span()[1]
answer_start = re.search(r"Answer:", output, re.DOTALL).span()[0]
answer_end = re.search(r"Answer:", output, re.DOTALL).span()[1]
coords_start = re.search(r'coords', output, re.DOTALL).span()[0]
coords_end = re.search(r'coords', output, re.DOTALL).span()[1]


In [ ]:

json.load(output)

In [ ]:
data.keys()

In [ ]:
Image.open(img_path  + '73003.png')


In [ ]:
messages = [system_prompt]
test_img = encode_image(img_path_)
messages.append({
    "role": "user",
    "parts": [
        {"mime_type": "image/jpeg", "data": test_img},
        {"text": "Now describe the map. And also give step-by-step reasoning for that answer. Explain step-by-step how you reach to the description and coordinates of map elements. "}
    ]
})
response = model.generate_content(messages)
output_ = response.text

output_


In [ ]:
output = output_
output = output.replace('"', "")
output = output.replace('*', "")
output = output.replace('```json', "")
output = output.replace('```', "")

print(output)

In [ ]:

reason_start = re.search(r'Reason', output, re.DOTALL).span()[0]
reason_end = re.search(r'Reason', output, re.DOTALL).span()[1]
answer_start = re.search(r'Answer', output, re.DOTALL).span()[0]
answer_end = re.search(r'Answer', output, re.DOTALL).span()[1]
coords_start = re.search(r'coords', output, re.DOTALL).span()[0]
coords_end = re.search(r'coords', output, re.DOTALL).span()[1]

reason_text = output[reason_end:answer_start]
reason_text = reason_text.replace("[", "")
reason_text = reason_text.replace("]", "")
reason_text = reason_text.replace("'", "")
reason_text = reason_text.replace(":", "")

reason_lines = [line.strip() for line in reason_text.strip().split("\n") if line.strip()]


answer_text = output[answer_end:coords_start]
answer_str = answer_text.replace("Answer:", "").strip()
answer_text = answer_text.replace("[", "")
answer_text = answer_text.replace("]", "")
answer_text = answer_text.replace("'", "")
answer_text = answer_text.replace(":", "")

coords_text = output[coords_end:]
coords_str = coords_text.replace("Answer:", "").strip()

result = {
        
        "Reason": reason_lines,
        "Answer": answer_str,
        "coords": coords_str,
    }


In [ ]:
result